# Chapter 7: Linked Lists

* a singly linked list is a data structure that contains a sequence of nodes such that each node contains an object and a reference to the next node in the list
* head - first node
* tail - last node (next field is null)
* doublely linked lists have a pointer to the preceeding node as well as the next node

**Complexity Considerations**
* Inserting and Deleting elements has time complexity O(1)
* Obtaining the kth element has time complexity O(n)

**Linked List Boot Camp**
* Basic linked list API:
    * search_list
    * insert_after
    * delete_after
* while insert and delete are O(1) searching for the node after which to insert delete can be O(n)


* List problems often have a brute-force O(n) space solution and a subtler O(1) solution
* Problems are often conceptually simple but more about cleaning coding 
* Consider using a dummy head (sentinel) to avoid having to check for empty lists. This simplifies code and makes bugs less likely
* It's easy to forget to update next for the head and tail
* Algorithms on singly linked lists often benefit from using 2 iterators, one ahead of the other, or one advancing quicker than the other

problems to do: 7.1, 7.2, 7.3, 7.4 done/continue here ->  7.7, 7.10, 7.11

Prereq: Build a linked list node and linked list class with some basics

In [1]:
class ListNode:
    def __init__(self, data=0, next_node=None):
        self.data = data
        self.next = next_node
    
    def get_next(self):
        return self.next
    
    def set_next(self,next_node):
        self.next = next_node
    
    def get_data(self):
        return self.data

In [2]:
class LinkedList():
    def __init__(self, head=None):
        self.head = head
        self.tail = head
    
    @classmethod
    def from_list(cls, input_list):
        """Instantiate linked list from regular list."""
        llist = cls()
        for value in reversed(input_list):
            llist.insert_at_head(ListNode(value))
        return llist
    
    def to_list(self):
        """Return linked list as regular list. O(N) operation."""
        ret = []
        next_node = self.get_head()
        while next_node:
            ret.append(next_node.get_data())
            next_node = next_node.get_next()
        return ret
    
    def insert_at_head(self, new_node):
        if self.head:
            new_node.set_next(self.get_head())
        self.set_head(new_node)
        if not self.head.get_next():
            self.set_tail(self.get_head())
    
    def insert_at_tail(self, new_node):
        if self.tail:
            self.get_tail().set_next(new_node)
            self.set_tail(new_node)
        else:
            self.set_head(new_node)
            self.set_tail(new_node)
        
    def get_head(self):
        return self.head
    
    def set_head(self, new_head):
        self.head = new_head
    
    def get_tail(self):
        return self.tail
    
    def set_tail(self, new_tail):
        self.tail = new_tail

## 7.1 Merge Two Sorted Lists
Write a program that takes 2 lists, assumed to be sorted (ascending), and returns their merge. The only field your program can change in a node is its next field

In [3]:
def merge_sorted(l1, l2):
    dummy_head = current = LinkedList(ListNode(0))
    l1_next = l1.get_head()
    l2_next = l2.get_head()
    
    while l1_next and l2_next:
        if l1_next.get_data() >= l2_next.get_data():
            current.next, l2_next = l2_next, l2_next.next
        else:
            current.next, l1_next = l1_next, l1_next.next
        current = current.next
    current.next = l1_next or l2_next
    return LinkedList(dummy_head.next)

assert(merge_sorted(LinkedList.from_list([2,5,7]), LinkedList.from_list([3,11])).to_list() == [2, 3, 5, 7, 11])
assert(merge_sorted(LinkedList.from_list([2,5,7,7,9,11,13]), 
                    LinkedList.from_list([3,5,9,11])).to_list() == [2, 3, 5, 5, 7, 7, 9, 9, 11, 11, 13])
assert(merge_sorted(LinkedList.from_list([2,5,7]), LinkedList.from_list([1,3,11])).to_list() == [1,2, 3, 5, 7, 11])

## 7.2 Reverse a single sublist
Write a program which takes a singly linked list L and 2 integers s and f as arguments, and reverses the order of the nodes from the sth node to the fth node, inclusive. Numbering begins at 1, i.e., the head node is the first node. Do not allocate additional nodes.

In [4]:
def reverse_sublist(L, s, f):
    """
    L is a singly linked list
    s is an int indicating the starting node in the sublist to reverse
    f is an int indicating the ending node in the sublist to reverse
    return the full linked list with the sublist reversed
    """
    dummy_head = pre_node = ListNode(0,L.head)
    for i in range(1,s):
        pre_node = pre_node.next
    start_node = pre_node.next
    for i in range(s,f):
        tmp = start_node.next
        start_node.next, tmp.next, pre_node.next = (
            tmp.next, pre_node.next, tmp)
    return LinkedList(dummy_head.next)
      

assert(LinkedList.to_list(reverse_sublist(LinkedList.from_list([1,2,3,4,5,6,7,8]),2,5)) == [1, 5, 4, 3, 2, 6, 7, 8])
assert(LinkedList.to_list(reverse_sublist(LinkedList.from_list([1,2,3,4,5,6,7,8]),1,5)) == [5, 4, 3, 2, 1, 6, 7, 8])
assert(LinkedList.to_list(reverse_sublist(LinkedList.from_list([1,2,3,4,5,6,7,8,9,10,11]),5,11)) == [1, 2, 3, 4, 11, 10, 9, 8, 7, 6, 5])
    

## 7.3 Test for Cyclicity
Write a program that takes the head of a singly linked list and returns null if there does not exist a cycle, and the node at the start of the cycle, if a cycle is present

In [21]:
def find_cycle_length(n1):
    """
    n1: a node in the cycle
    return: length of the cycle
    """
    n1_move = n1.next
    i = 1
    while n1_move != n1:
        i += 1
        n1_move = n1_move.next
    return i
    
def detect_cycles(L):
    slow = L.head
    fast = L.head.next.next
    cycle = False
    i = 0
    while slow and fast.next and fast.next.next:
        i += 1
        #print slow.data
        #print fast.data
        slow = slow.next
        fast = fast.next.next
        if slow == fast:
            cycle = True
            break
    if not cycle:
        return None
    cycle_len = find_cycle_length(slow)
    fast = L.head
    for i in range(0,cycle_len):
        fast = fast.next
    slow = L.head
    while slow != fast:
        slow = slow.next
        fast = fast.next
    return fast.data

ll = LinkedList.from_list([1,2,3,4,5])
assert(detect_cycles(ll) == None)

ll = LinkedList.from_list([1,2,3,4,5])
ll.insert_at_tail(ll.head)
assert(detect_cycles(ll) == 1)

ll = LinkedList.from_list([1,2,3,4,5])
ll.insert_at_tail(ll.head.next)
assert(detect_cycles(ll) == 2)

ll = LinkedList.from_list([1,2,3,4,5])
ll.insert_at_tail(ll.head.next.next)
assert(detect_cycles(ll) == 3)